In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:08<00:00, 1134.40it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("bin")


{'basic': {'expected_sample_burst_size': 3.002660148105543,
           'expected_sample_data_size': 16.52606226184485,
           'expected_time_to_recover': 99.3083,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16111442850194796,
             'bit_redundancy': 416.54150000000004,
             'ideal_bit_efficency': 0.19038769796460303,
             'ideal_bit_redundancy': 340.19521668292685,
             'ideal_packet_efficiency': 0.9519384898230152,
             'ideal_packet_redundancy': 68.03904333658537,
             'ideal_time_to_recover': 84.03904333658537,
             'ideal_transmitted_bits': 420.19521668292685,
             'packet_efficiency': 0.8055721425097399,
             'packet_redundancy': 83.3083,
             'permeability': 0.8462439024390244,
             'saturation': 16.0,
             'transmitted_bits': 496.54150000000004}}
Found implementation: anfext_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:09<00:00, 1078.49it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



{'basic': {'expected_sample_burst_size': 2.9972967505838652,
           'expected_sample_data_size': 16.5031721804373,
           'expected_time_to_recover': 96.8144,
           'packet_size': 5,
           'payload_bits': 78.25340432877415},
 'derived': {'bit_efficency': 0.1616565393759072,
             'bit_redundancy': 405.81859567122586,
             'ideal_bit_efficency': 0.1910165081676283,
             'ideal_bit_redundancy': 331.4148755462938,
             'ideal_packet_efficiency': 0.9550825408381415,
             'ideal_packet_redundancy': 66.28297510925876,
             'ideal_time_to_recover': 81.9336559750136,
             'ideal_transmitted_bits': 409.6682798750679,
             'packet_efficiency': 0.808282696879536,
             'packet_redundancy': 81.16371913424517,
             'permeability': 0.8462961705594786,
             'saturation': 15.65068086575483,
             'transmitted_bits': 484.072}}
Found implementation: shuffled_binary_fullrank
  First time seen. B

...

{'basic': {'expected_sample_burst_size': 2.9971133462631365,
           'expected_sample_data_size': 16.60281448739344,
           'expected_time_to_recover': 33.278,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.1502494140272853,
             'bit_redundancy': 141.39,
             'ideal_bit_efficency': 0.17737219639598054,
             'ideal_bit_redundancy': 115.94655480382002,
             'ideal_packet_efficiency': 0.8868609819799026,
             'ideal_packet_redundancy': 23.189310960764008,
             'ideal_time_to_recover': 28.189310960764008,
             'ideal_transmitted_bits': 140.94655480382002,
             'packet_efficiency': 0.7512470701364264,
             'packet_redundancy': 28.278,
             'permeability': 0.8470854907375446,
             'saturation': 5.0,
             'transmitted_bits': 166.39}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3595.79it/s]


{'basic': {'expected_sample_burst_size': 3.0174321420042984,
           'expected_sample_data_size': 16.573987104990845,
           'expected_time_to_recover': 13.3238,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3490506361364499,
             'bit_redundancy': 43.36559567122585,
             'ideal_bit_efficency': 0.41259820631333355,
             'ideal_bit_redundancy': 33.105067358606746,
             'ideal_packet_efficiency': 2.0629910315666677,
             'ideal_packet_redundancy': 6.62101347172135,
             'ideal_time_to_recover': 11.27169433747618,
             'ideal_transmitted_bits': 56.3584716873809,
             'packet_efficiency': 1.745253180682249,
             'packet_redundancy': 8.67311913424517,
             'permeability': 0.8459819524066843,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.619}}
Found implementation: single_binary_fullrank2
  First time seen. B

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3472.82it/s]


{'basic': {'expected_sample_burst_size': 2.9714688822699484,
           'expected_sample_data_size': 16.46645242200972,
           'expected_time_to_recover': 13.836500000000001,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3361168551118296,
             'bit_redundancy': 45.92909567122585,
             'ideal_bit_efficency': 0.39677113268021796,
             'ideal_bit_redundancy': 35.35318877616196,
             'ideal_packet_efficiency': 1.98385566340109,
             'ideal_packet_redundancy': 7.07063775523239,
             'ideal_time_to_recover': 11.72131862098722,
             'ideal_transmitted_bits': 58.60659310493611,
             'packet_efficiency': 1.680584275559148,
             'packet_redundancy': 9.18581913424517,
             'permeability': 0.8471303162640278,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.1825}}
